## Importing libraries and modules

In [1]:
# Enable auto-reload
%load_ext autoreload
%autoreload 2

In [2]:
# Suppress warnings
import warnings
warnings.filterwarnings("ignore")

# Add one level up directory to the path
import sys
sys.path.append("..")

# Import libraries
import torch
import random
import pandas as pd

# Import custom modules
from src.data import *
from src.machine_translation import *
from src.data.utils import get_dataset
from src.data.preprocess import clean_text
from src.data.tokenizer import CustomBartTokenizer
from src.machine_translation.translate import translate
from src.machine_translation.net import CodeMixedModel, CodeMixedModelHGTrainer
from src.machine_translation.models.bart_conditional import BartForConditionalGeneration
from src.machine_translation.utils import get_tokenized_dataset_models, get_data_loader_models, calculate_sacrebleu_score, calculate_chrf_score, calculate_bert_score, calculate_tokens

## Data acquisition, cleaning and processing

In [3]:
%%time
train_df, validation_df, test_df = get_dataset()

CPU times: user 183 ms, sys: 16.7 ms, total: 200 ms
Wall time: 200 ms


In [4]:
train_df

,hi_en,en,source
0,film ka kya naam hai,What's the name of the movie,cmu_hinglish_dog
1,"namaste, sada hua tomatoes score mahaan hai, l...","Hi, the rotten tomatoes score is great but the...",cmu_hinglish_dog
2,kya aapako lagata hai ki aapako film pasand aa...,Do you think you will like the movie,cmu_hinglish_dog
3,yah kis tarah kee philm hai,What kind of movie is it,cmu_hinglish_dog
4,film kab banee thee?,when was the movie made?,cmu_hinglish_dog
...,...,...,...
174438,Thik hai\n,Ok.,linc
174439,Thik hai bhai\n,ok bro,linc
174440,Kya ham chalu kar sakte hai?\n,shall we continue?,linc
174441,Kya aapko pasand hai hamare saath\n,do you like we can,linc


In [5]:
validation_df

,hi_en,en,source
0,movie kis baare me hai?,What is the movie about?,cmu_hinglish_dog
1,Movie ek chhote bacche Kevin k baare me hai ji...,the movie is about a young child named Kevin w...,cmu_hinglish_dog
2,Kya wo jaan bhuj k abandon karte hai?,Did they abandon him on purpose?,cmu_hinglish_dog
3,"nahi, wo uska track lose kardete hai kyuki bah...",no they had lost track of him since they had m...,cmu_hinglish_dog
4,Kya wo realize karte hai k wo chhut gaya aur u...,Did they realize they lost track of him and co...,cmu_hinglish_dog
...,...,...,...
3238,Sunkar good movie hai. Kya ham finish kar skat...,Sounds like a good movie. Can we finish now?,linc
3239,Yep. Thanks baat karne ke liye\n,Yep. Thanks for chatting,linc
3240,"Thanks, mei dekhati hui. Achi baat hai\n","thanks, I will watch it. SOunds good",linc
3241,kya tumhe movie Despicable Me pasand hai?\n,Did you like the movie Despicable Me?,linc


In [6]:
test_df

,hi_en,en,source
0,oye ledki kitni mazakiya movie dekh rhi ho ?,ah mean girls. such a funny movie. have you se...,cmu_hinglish_dog
1,ha muje bhi hassi mazak ki movies pasand par y...,"Yeah, even though I love comedies, this wasn't...",cmu_hinglish_dog
2,kese tina ka jhuti writing isme achi the usne ...,how come. thought tina fey's writing was fanta...,cmu_hinglish_dog
3,muje kahani bhute achi lagi aur unhone ise dac...,I loved the story & how true they made it in h...,cmu_hinglish_dog
4,ab samja muje nhi pta tha ki aap sahi admi hai...,I see. i didn't realize it was 14 years ago. y...,cmu_hinglish_dog
...,...,...,...
7067,alarm ko abhi stop kare,Stop alarm now,top
7068,Har ghante ke liye alarm set kare,set alarm every hour,top
7069,Bobby ko text kare,text Bobby,top
7070,Muje shaam 6 baje laundry ko pick up karne ke ...,remind me to pick up laundry at 6 pm,top


In [7]:
%%time
# Cleaning train_df
train_df = train_df.applymap(clean_text)
train_df

CPU times: user 2.83 s, sys: 12.1 ms, total: 2.84 s
Wall time: 2.85 s


,hi_en,en,source
0,film ka kya naam hai,what's the name of the movie,cmu_hinglish_dog
1,"namaste, sada hua tomatoes score mahaan hai, l...","hi, the rotten tomatoes score is great but the...",cmu_hinglish_dog
2,kya aapako lagata hai ki aapako film pasand aa...,do you think you will like the movie,cmu_hinglish_dog
3,yah kis tarah kee philm hai,what kind of movie is it,cmu_hinglish_dog
4,film kab banee thee?,when was the movie made?,cmu_hinglish_dog
...,...,...,...
174438,thik hai,ok.,linc
174439,thik hai bhai,ok bro,linc
174440,kya ham chalu kar sakte hai?,shall we continue?,linc
174441,kya aapko pasand hai hamare saath,do you like we can,linc


In [8]:
%%time
# Cleaning validation_df
validation_df = validation_df.applymap(clean_text)
validation_df

CPU times: user 52.9 ms, sys: 947 µs, total: 53.9 ms
Wall time: 53.5 ms


,hi_en,en,source
0,movie kis baare me hai?,what is the movie about?,cmu_hinglish_dog
1,movie ek chhote bacche kevin k baare me hai ji...,the movie is about a young child named kevin w...,cmu_hinglish_dog
2,kya wo jaan bhuj k abandon karte hai?,did they abandon him on purpose?,cmu_hinglish_dog
3,"nahi, wo uska track lose kardete hai kyuki bah...",no they had lost track of him since they had m...,cmu_hinglish_dog
4,kya wo realize karte hai k wo chhut gaya aur u...,did they realize they lost track of him and co...,cmu_hinglish_dog
...,...,...,...
3238,sunkar good movie hai. kya ham finish kar skat...,sounds like a good movie. can we finish now?,linc
3239,yep. thanks baat karne ke liye,yep. thanks for chatting,linc
3240,"thanks, mei dekhati hui. achi baat hai","thanks, i will watch it. sounds good",linc
3241,kya tumhe movie despicable me pasand hai?,did you like the movie despicable me?,linc


In [9]:
%%time
# Cleaning test_df
test_df = test_df.applymap(clean_text)
test_df

CPU times: user 107 ms, sys: 1.21 ms, total: 108 ms
Wall time: 108 ms


,hi_en,en,source
0,oye ledki kitni mazakiya movie dekh rhi ho ?,ah mean girls. such a funny movie. have you se...,cmu_hinglish_dog
1,ha muje bhi hassi mazak ki movies pasand par y...,"yeah, even though i love comedies, this wasn't...",cmu_hinglish_dog
2,kese tina ka jhuti writing isme achi the usne ...,how come. thought tina fey's writing was fanta...,cmu_hinglish_dog
3,muje kahani bhute achi lagi aur unhone ise dac...,i loved the story & how true they made it in h...,cmu_hinglish_dog
4,ab samja muje nhi pta tha ki aap sahi admi hai...,i see. i didn't realize it was 14 years ago. y...,cmu_hinglish_dog
...,...,...,...
7067,alarm ko abhi stop kare,stop alarm now,top
7068,har ghante ke liye alarm set kare,set alarm every hour,top
7069,bobby ko text kare,text bobby,top
7070,muje shaam 6 baje laundry ko pick up karne ke ...,remind me to pick up laundry at 6 pm,top


## Data tokenization

In [10]:
%%time
bart_tokenizer = CustomBartTokenizer().build()
bart_tokenizer_scratch = CustomBartTokenizer().build(data=train_df["hi_en"], tokenizer_style=STYLE.SCRATCH.value)
bart_tokenizer_append = CustomBartTokenizer().build(data=train_df["hi_en"], tokenizer_style=STYLE.APPEND.value)







CPU times: user 27.8 s, sys: 80.6 ms, total: 27.9 s
Wall time: 28.4 s


In [11]:
query = "Hello! Ye CSCI 544 ka project he. Ye project Code-Mixed Machine Translation ke bara me he. Hamane hamari khoon bahahe."
query = clean_text(text=query)

In [12]:
token_idx = bart_tokenizer.encode(query, add_special_tokens=True)
token_word = bart_tokenizer.convert_ids_to_tokens(token_idx)
print('#'*100)
print("This is an example of using the default Bart Tokenizer.\n")
print(f"Sentence: {query}\n")
print(f"Tokenized word: {token_word}\n")
print(f"Tokenized idx: {token_idx}")
print('#'*100)

####################################################################################################
This is an example of using the default Bart Tokenizer.

Sentence: hello! ye csci 544 ka project he. ye project code-mixed machine translation ke bara me he. hamane hamari khoon bahahe.

Tokenized word: ['<s>', 'hello', '!', 'Ġye', 'Ġc', 'sci', 'Ġ5', '44', 'Ġka', 'Ġproject', 'Ġhe', '.', 'Ġye', 'Ġproject', 'Ġcode', '-', 'm', 'ixed', 'Ġmachine', 'Ġtranslation', 'Ġke', 'Ġbar', 'a', 'Ġme', 'Ġhe', '.', 'Ġham', 'ane', 'Ġham', 'ari', 'Ġkh', 'oon', 'Ġb', 'aha', 'he', '.', '</s>']

Tokenized idx: [0, 42891, 328, 32440, 740, 43428, 195, 3305, 4661, 695, 37, 4, 32440, 695, 3260, 12, 119, 24194, 3563, 19850, 7321, 2003, 102, 162, 37, 4, 11402, 1728, 11402, 1512, 16447, 3863, 741, 11695, 700, 4, 2]
####################################################################################################


In [13]:
token_idx = bart_tokenizer_scratch.encode(query, add_special_tokens=True)
token_word = bart_tokenizer_scratch.convert_ids_to_tokens(token_idx)
print('#'*100)
print("This is an example of the Bart Tokenizer that is trained from scratch.\n")
print(f"Sentence: {query}\n")
print(f"Tokenized word: {token_word}\n")
print(f"Tokenized idx: {token_idx}")
print('#'*100)

####################################################################################################
This is an example of the Bart Tokenizer that is trained from scratch.

Sentence: hello! ye csci 544 ka project he. ye project code-mixed machine translation ke bara me he. hamane hamari khoon bahahe.

Tokenized word: ['<s>', 'hello', '!', 'Ġye', 'Ġc', 'sci', 'Ġ5', '44', 'Ġka', 'Ġproject', 'Ġhe', '.', 'Ġye', 'Ġproject', 'Ġcode', '-', 'mix', 'ed', 'Ġmachine', 'Ġtrans', 'lation', 'Ġke', 'Ġbara', 'Ġme', 'Ġhe', '.', 'Ġhamane', 'Ġhamari', 'Ġkhoon', 'Ġbaha', 'he', '.', '</s>']

Tokenized idx: [0, 2317, 4, 520, 288, 21554, 466, 11342, 312, 3070, 356, 17, 520, 3070, 6438, 16, 19530, 517, 6262, 3397, 18402, 271, 17190, 284, 356, 17, 11714, 4630, 10623, 13967, 282, 17, 2]
####################################################################################################


In [14]:
token_idx = bart_tokenizer_append.encode(query, add_special_tokens=True)
token_word = bart_tokenizer_append.convert_ids_to_tokens(token_idx)
print('#'*100)
print("This is an example of the Bart Tokenizer to which we append our new data.\n")
print(f"Sentence: {query}\n")
print(f"Tokenized word: {token_word}\n")
print(f"Tokenized idx: {token_idx}")
print('#'*100)

####################################################################################################
This is an example of the Bart Tokenizer to which we append our new data.

Sentence: hello! ye csci 544 ka project he. ye project code-mixed machine translation ke bara me he. hamane hamari khoon bahahe.

Tokenized word: ['<s>', 'hello', '!', 'Ġ', 'ye', 'Ġ', 'csc', 'i', 'Ġ', '54', '4', 'Ġ', 'ka', 'Ġ', 'project', 'Ġ', 'he', '.', 'Ġ', 'ye', 'Ġ', 'project', 'Ġ', 'code', '-', 'mix', 'ed', 'Ġ', 'mach', 'ine', 'Ġ', 'trans', 'lation', 'Ġ', 'ke', 'Ġ', 'bara', 'Ġ', 'me', 'Ġ', 'he', '.', 'Ġ', 'hamane', 'Ġ', 'hamari', 'Ġ', 'khoo', 'n', 'Ġ', 'baha', 'he', '.', '</s>']

Tokenized idx: [0, 42891, 328, 1437, 4717, 1437, 57353, 118, 1437, 4283, 306, 1437, 2348, 1437, 28258, 1437, 700, 4, 1437, 4717, 1437, 28258, 1437, 20414, 12, 39915, 196, 1437, 68816, 833, 1437, 9981, 35019, 1437, 1071, 1437, 31533, 1437, 1794, 1437, 700, 4, 1437, 55446, 1437, 52754, 1437, 81856, 282, 1437, 52064, 700, 4, 2]
########

In [15]:
%%time
# Now that we know how to get a BART-Tokenizer - default, appeded and scratch, let us get the tokenizer for our code-mixed language
# and the target language.
# Code-Mixed Language - Build a Bart-Tokenizer from scratch.
# Target Language (English) - Use the default Bart-Tokenizer.
hi_en_bart_tokenizer = CustomBartTokenizer().build(
    data=train_df["hi_en"],
    tokenizer_style=MBART_TOKENIZER_BPE_BINDING_BART_TOKENIZER_ENCODER_STYLE,
    tokenizer_bart_from_pretrained_path=MBART_TOKENIZER_BPE_BINDING_BART_TOKENIZER_ENCODER_FROM_PRETRAINED
)
en_bart_tokenizer = CustomBartTokenizer().build(
    tokenizer_style=MBART_TOKENIZER_BPE_BINDING_BART_TOKENIZER_DECODER_STYLE,
    tokenizer_bart_from_pretrained_path=MBART_TOKENIZER_BPE_BINDING_BART_TOKENIZER_DECODER_FROM_PRETRAINED
)




CPU times: user 13.6 s, sys: 34.8 ms, total: 13.7 s
Wall time: 13.9 s


In [16]:
print(f"Hinglish tokenizer vocab size: {hi_en_bart_tokenizer.vocab_size}")
print(f"English tokenizer vocab size: {en_bart_tokenizer.vocab_size}")

Hinglish tokenizer vocab size: 50265
English tokenizer vocab size: 50265


In [17]:
%%time
print(f"There are {train_df['hi_en'].apply(lambda x: calculate_tokens(x, hi_en_bart_tokenizer)).sum()} hinglish tokens in the train data.")
print(f"There are {train_df['en'].apply(lambda x: calculate_tokens(x, en_bart_tokenizer)).sum()} english tokens in the train data.")
print(f"There are {validation_df['hi_en'].apply(lambda x: calculate_tokens(x, hi_en_bart_tokenizer)).sum()} hinglish tokens in the validation data.")
print(f"There are {validation_df['en'].apply(lambda x: calculate_tokens(x, en_bart_tokenizer)).sum()} english tokens in the validation data.")
print(f"There are {test_df['hi_en'].apply(lambda x: calculate_tokens(x, hi_en_bart_tokenizer)).sum()} hinglish tokens in the test data.")
print(f"There are {test_df['en'].apply(lambda x: calculate_tokens(x, en_bart_tokenizer)).sum()} english tokens in the test data.")

There are 1764738 hinglish tokens in the train data.
There are 1807859 english tokens in the train data.
There are 38502 hinglish tokens in the validation data.
There are 38366 english tokens in the validation data.
There are 70895 hinglish tokens in the test data.
There are 69686 english tokens in the test data.
CPU times: user 9.02 s, sys: 7.63 ms, total: 9.03 s
Wall time: 9.05 s


## Dataset and Data Loaders

In [18]:
# Get the tokenized dataset
__tokenized_datasets__ = get_tokenized_dataset_models(
    train_df=train_df,
    validation_df=validation_df,
    test_df=test_df,
    encoder_tokenizer=hi_en_bart_tokenizer,
    decoder_tokenizer=en_bart_tokenizer
)
__tokenized_datasets__

defaultdict(dict,
            {'hi_en__en': {'train': <src.machine_translation.data.CodeMixedTokenizedDataset at 0x29d1a57c0>,
              'validation': <src.machine_translation.data.CodeMixedTokenizedDataset at 0x29d1a5eb0>,
              'test': <src.machine_translation.data.CodeMixedTokenizedDataset at 0x29d1a5ee0>}})

In [19]:
__train_dataset__, __validation_dataset__, __test_dataset__ = __tokenized_datasets__["hi_en__en"].values()

In [20]:
__train_dataset__.visualize()

####################################################################################################
Dataset
Number of instances:  174443
Denoising stage:  False
Source language:  hi_en
Target language:  en
Key: input_ids, Value: tensor([    0,  3497,    15, 10386,  1071,  2025,  2185,  6089,   278,    15,
         1088,  4175,  2430,  2185,   392, 33673,  1711,  1963,   291,  1104,
         1067,  1758,   278,    17,     2,     1,     1,     1,     1,     1]), Value shape: torch.Size([30])
Key: attention_mask, Value: tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 0, 0, 0, 0, 0]), Value shape: torch.Size([30])
Key: labels, Value: tensor([    0,  3592,     6,     5, 34485, 18553,  1471,    16,   372,    53,
            5, 32820,  7745,  1471,  1302,    10,   410,   614,    10,  1569,
            9,    42,  1318,     4,     2,  -100,  -100,  -100,  -100,  -100]), Value shape: torch.Size([30])
####################################################

In [21]:
# Get the data loaders for denoising
__data_loaders__ = get_data_loader_models(
    train_df=train_df,
    validation_df=validation_df,
    test_df=test_df,
    train_batch_size=8, 
    validation_batch_size=8, 
    test_batch_size=8,
    encoder_tokenizer=hi_en_bart_tokenizer,
    decoder_tokenizer=hi_en_bart_tokenizer,
    denoising_stage=True
)

In [22]:
__data_loaders__

defaultdict(dict,
            {'hi_en__en': {'object': <src.machine_translation.data.CodeMixedDataLoader at 0x29d2cabb0>,
              'train': <torch.utils.data.dataloader.DataLoader at 0x29d29b070>,
              'validation': <torch.utils.data.dataloader.DataLoader at 0x29d1a57f0>,
              'test': <torch.utils.data.dataloader.DataLoader at 0x29e5ffee0>}})

In [23]:
__data_loader__, __train_data_loader__, __validation_data_loader__, __test_data_loader__ = __data_loaders__["hi_en__en"].values()

In [24]:
__data_loader__.visualize()

####################################################################################################
Train Dataloader
Batch Size:  8
Number of batches:  21806
Batch source language shape:  torch.Size([8, 30])
Batch source language:  ['hamare pass aaj kitne centimetres ki baarish hui hai ?', 'palm beach me kya koi christmas events hai', 'los angeles me agla justin timberlake ka concert kab hai', 'mujhe friday 3rd of april 2 pm ke liye weather forecast do', 'current song ko wapas play kariye', 'modiji duniya k masiha he.kafi ku6 kiya he or aage bhi karna he.', 'kya aap staff ko expense reports bhejne ke liye monthly ke last day ke liye ek monthly reminder set up kar sakte hai ?', 'so the fact that when hum baahar x ki hota yah pata lagane ki koshis ki.']
Batch source tokens:  tensor([[    0,  2588, 50265,   439,   551, 11989,   294,   707,  1763,   278,
           318,     2,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,

100%|█| 21806/21806 [00:41<00:00, 526


Validation of train loader successful.
####################################################################################################
Val Dataloader
Batch Size:  8
Number of batches:  406
Validating validation laoder...


100%|█| 406/406 [00:32<00:00, 12.45it


Validation of validation loader successful.
####################################################################################################
Test Dataloader
Batch Size:  8
Number of batches:  884
Validating test laoder...


100%|█| 884/884 [00:32<00:00, 26.99it

Validation of test loader successful.
####################################################################################################
